In [2]:
#----------------------------------------------------------------------------------------#
#
# This program graphs, normalizes and convolutes spectra from data given
# in POPUVES (https://www.eso.org/sci/observing/tools/uvespop.html) 
# In this case, the problem spectra have been already normalized, so we
# will just plot them.
#
# Copyright (C) 2021 Andrés García-Serra Romero <alu0101451923@ull.edu.es>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.





##IMPORTING##
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy import units as u
from astropy.visualization import quantity_support
from specutils import Spectrum1D
from specutils.manipulation import (box_smooth, gaussian_smooth, trapezoid_smooth, median_smooth)
from specutils.fitting import fit_generic_continuum
import coronagraph as cg





##PLOT-FORMAT##
plt.rc('font', size=20)          # controls default text sizes
plt.rc('axes', titlesize=20)     # fontsize of the axes title
plt.rc('axes', labelsize=20)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels




        
##PLOTTING-STD##
# Creating arrays to classify fluxes:
stand_stars = {}
files = os.listdir(".")
for name in files:
    if name[-3:] == "dat":
        if name[:2] == "hd":
            
            # Importing data:
            data = np.loadtxt(name,skiprows=1,delimiter="  ")
            f = open(name, 'r')
            startype = f.readline().lower()
            startype = startype[:-1]
            lamda = [i[0] for i in data]
            flux = [i[1] for i in data]
              
            # Normalizing:
            flux = np.array(flux)
            lamda = np.array(lamda)
            flux = flux[(lamda<5000) & (lamda>3900)]
            lamda = lamda[(lamda<5000) & (lamda>3900)]
            flux = flux [5:]
            lamda = lamda [5:]
            y_continuum = 3.2 * np.exp(-0.5 * (lamda - 5.6)**2 / 4.8**2)
            flux += y_continuum
            spectrum = Spectrum1D(flux=flux*u.Jy, spectral_axis=lamda*u.AA)
            g1_fit = fit_generic_continuum(spectrum)
            y_continuum_fitted = g1_fit(lamda*u.AA)
            spec_normalized = spectrum / y_continuum_fitted
            stand_stars[startype] = spec_normalized
            
            
        if name[:2] == "st":
                
            # Importing data:
            data = np.loadtxt(name,skiprows=1,delimiter="  ")
            f = open(name, 'r')
            lamda = [i[0] for i in data]
            flux = [i[1] for i in data]
            flux = np.array(flux)
            lamda = np.array(lamda)
            flux = flux[(lamda < 5000)&(lamda > 3900)]
            lamda = lamda[(lamda < 5000)&(lamda > 3900)]
        
            # Resolution change:
            lammin = 3900
            lammax = 5000
            nwl, dwl = cg.noise_routines.construct_lam(lammin, lammax, Res=5000)
            nwf = cg.downbin_spec(flux,lamda,nwl,dlam=dwl)

    """# Plotting:
    plt.figure(figsize=(15,5))
    plt.plot(nwl,nwf,"k-",label="Spectrum")
    plt.title(name[:-4])
    plt.ylabel("Normalized Flux")
    plt.xlabel(r"$\lambda$"+r"$(\AA)$")
    plt.savefig("spectra/"+name[:-4]+".png")"""
    

stand_stars.keys()

# ASÍ ES COMO SE LLAMA A LAS VARIABLES DENTRO DEL "DICCIONARIO" STAND_STARS:
print(stand_stars['g2v'].flux) 
print(stand_stars['g2v'].spectral_axis) 
        

UnitTypeError: SpectralCoord instances require units equivalent to '(Unit("Hz"), Unit("m"), Unit("J"), Unit("1 / m"), Unit("km / s"))', so cannot set it to ''.

17395
